In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import sys
sys.path.append("../helper_functions")
import duneapi_utils as du
sys.path.pop()

# Define the CSV path and API URL
csv_path = 'outputs/daily_op_circulating_supply.csv'  # Update this path as per your directory structure
api_url = 'https://static.optimism.io/tokenomics/circulatingSupply.txt'

In [2]:
def update_csv_with_api_data(csv_path, api_url):
    # Read the existing CSV file
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"File {csv_path} not found.")
        return

    # Get the current data from the API
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        current_supply = float(response.text.strip())
    except requests.RequestException as e:
        print(f"Error fetching data from API: {e}")
        return

    # Get today's date in the required format
    today = datetime.now().strftime('%Y-%m-%d')

    # Check if today's date is already in the DataFrame
    if today in df['date'].values:
        df.loc[df['date'] == today, 'op_circulating_supply'] = current_supply
    else:
        # Backfill missing dates
        last_date_in_csv = datetime.strptime(df['date'].iloc[-1], '%Y-%m-%d')
        delta = datetime.strptime(today, '%Y-%m-%d') - last_date_in_csv
        for i in range(1, delta.days + 1):
            new_date = last_date_in_csv + timedelta(days=i)
            new_date_str = new_date.strftime('%Y-%m-%d')
            df = df.append({'date': new_date_str, 'op_circulating_supply': current_supply}, ignore_index=True)

    # Save the updated DataFrame back to CSV
    df.to_csv(csv_path, index=False)
    print(f"CSV file updated successfully at {csv_path}")

In [3]:
# Call the function to update the CSV
update_csv_with_api_data(csv_path, api_url)

CSV file updated successfully at outputs/daily_op_circulating_supply.csv


In [4]:
try:
    # Read the CSV file
    df = pd.read_csv(csv_path)

    # Print the last 10 rows
    print(df.tail(10))
except FileNotFoundError:
    print(f"File {csv_path} not found.")
except Exception as e:
    print(f"An error occurred: {e}")

           date  op_circulating_supply
561  2023-12-13            911294948.0
562  2023-12-14            911294948.0
563  2023-12-15            911294948.0
564  2023-12-16            911294948.0
565  2023-12-17            911294948.0
566  2023-12-18            911294948.0
567  2023-12-19            911294948.0
568  2023-12-20            911294948.0
569  2023-12-21            911294948.0
570  2023-12-22            911294948.0


In [5]:
# Convert to timestamp
df['date'] = pd.to_datetime(df['date'])
# Upload to Dune
du.write_dune_api_from_pandas(df, 'daily_op_circulating_supply',\
                             'Daily Snapshots of OP Token Circulating Supply, pulled from: https://static.optimism.io/tokenomics/circulatingSupply.txt')

table at: dune.oplabspbc.dataset_daily_op_circulating_supply
Response status code: 200
Response content: b'{"success":true,"table_name":"daily_op_circulating_supply"}'
